In [54]:
#kernel = use CMIP6 AWS
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
from scipy import stats
import matplotlib
import matplotlib.animation as animation
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import warnings
import matplotlib.path as mpath
import intake
import pandas as pd
import s3fs
import proplot as pplt
from numba import jit
import os
warnings.filterwarnings("ignore")

In [2]:
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')
#query = dict(variable_id = ["rsds","rsus","rlds","rlus","hfss","hfls"], table_id = "Amon",\
#             experiment_id = ['historical', 'ssp585'])
query = dict(variable_id = ["msftmz"],table_id="Omon",\
             experiment_id = ['piControl', 'abrupt-4xCO2'])
res = catalog.search(require_all_on=["source_id"],**query)
models = res.df['source_id'].unique()
print((res.df['source_id'].unique()))
#Other msft* variables aren't available, so this is probably the dataset to use.
#Decent collection of models

['ACCESS-CM2' 'ACCESS-ESM1-5' 'CESM2' 'CESM2-FV2' 'CESM2-WACCM'
 'CESM2-WACCM-FV2' 'CanESM5' 'E3SM-1-0' 'FGOALS-g3' 'GISS-E2-1-G'
 'GISS-E2-2-G' 'INM-CM4-8' 'MIROC6' 'MPI-ESM-1-2-HAM' 'MPI-ESM1-2-HR'
 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'SAM0-UNICON']


In [6]:
#set up dictionaries
nyr = 150
ntime = nyr*12
dict_piC_AMOC = {}

lats_AMOC = {}
depth_AMOC = {}

#set up AWS filesystem access
fs_s3 = s3fs.S3FileSystem(anon=True)
myvars = ["msftmz"] #(time, basin, lev, lat)

for imodel in enumerate(models):
    print("piControl, "+imodel[1])
    for ivar in range(0,len(myvars)):
        res = catalog.search(source_id=imodel[1], variable_id=myvars[ivar],\
                             experiment_id="piControl", table_id="Omon")
        #print(res.df['zstore'][0])
        mapper = fs_s3.get_mapper(res.df['zstore'][0])
        ds = xr.open_zarr(mapper, consolidated=True)
        #print(ds)
        try:
            lats_AMOC[imodel[1]] = ds.lat
            depth_AMOC[imodel[1]] = ds.lev
        except:
            print("Latitude or depth error")
        try:
            print(np.shape(ds[myvars[ivar]].isel(basin=0)))
            dict_piC_AMOC[imodel[1]+"_"+myvars[ivar]]=ds[myvars[ivar]].isel(basin=0)[0:ntime,:,:]
        except (AttributeError, KeyError):
            print(myvars[ivar]+" not found")

piControl, ACCESS-CM2
(6000, 50, 300)
piControl, ACCESS-ESM1-5
(12000, 50, 300)
piControl, CESM2
(14400, 61, 395)
piControl, CESM2-FV2
(6000, 61, 395)
piControl, CESM2-WACCM
(5988, 61, 395)
piControl, CESM2-WACCM-FV2
(6000, 61, 395)
piControl, CanESM5
(12000, 45, 290)
piControl, E3SM-1-0
(6000, 60, 180)
piControl, FGOALS-g3
(8388, 31, 316)
piControl, GISS-E2-1-G
(4212, 40, 180)
piControl, GISS-E2-2-G
(1812, 40, 180)
piControl, INM-CM4-8
(6372, 33, 181)
piControl, MIROC6
(9600, 61, 180)
piControl, MPI-ESM-1-2-HAM
(12000, 41, 180)
piControl, MPI-ESM1-2-HR
(6000, 41, 180)
piControl, MPI-ESM1-2-LR
(12000, 41, 180)
piControl, MRI-ESM2-0
(8412, 61, 362)
piControl, SAM0-UNICON
(8388, 61, 395)


In [7]:
for key in dict_piC_AMOC:
    print(np.shape(dict_piC_AMOC[key]))

(1800, 50, 300)
(1800, 50, 300)
(1800, 61, 395)
(1800, 61, 395)
(1800, 61, 395)
(1800, 61, 395)
(1800, 45, 290)
(1800, 60, 180)
(1800, 31, 316)
(1800, 40, 180)
(1800, 40, 180)
(1800, 33, 181)
(1800, 61, 180)
(1800, 41, 180)
(1800, 41, 180)
(1800, 41, 180)
(1800, 61, 362)
(1800, 61, 395)


In [10]:
AMOCindMax_piC = {}
AMOCindLoc_piC = {}
for i in enumerate(models):
    latmod = lats_AMOC[i[1]]
    depthmod = depth_AMOC[i[1]]
    latmesh, depthmesh = np.meshgrid(latmod, depthmod)
    #print(latmesh)
    #print(depthmesh)
    MOCmod_piC = dict_piC_AMOC[i[1]+"_msftmz"]
    MOCmod_piC = np.where(np.isnan(MOCmod_piC),0.,MOCmod_piC)
    if i[1]=="E3SM-1-0":
        MOCmod_piC = MOCmod_piC*1.e-6
    else:
        MOCmod_piC = MOCmod_piC*1.e-9
    myMask = np.where((latmesh>30.)&(latmesh<70.)&(depthmesh>300.)&(depthmesh<2000.), 1., 0.)
    myAMOCindMax = np.zeros([1800])
    for j in range(0,1800):
        myAMOCindMax[j] = np.max(MOCmod_piC[j,:,:]*myMask)
    AMOCindMax_piC[i[1]] = myAMOCindMax
    inds = np.unravel_index((np.mean(MOCmod_piC,axis=0)*myMask).argmax(),np.shape(MOCmod_piC[0,:,:]))
    myAMOCindLoc = np.zeros([1800])
    for j in range(0,1800):
        myAMOCj = MOCmod_piC[j,:,:]
        myAMOCindLoc[j] = myAMOCj[inds]
    AMOCindLoc_piC[i[1]]=myAMOCindLoc

In [12]:
timevec = np.linspace(0.,1799.,1800)
#detrend and remove seasonal cycle
for i in enumerate(models):
    myAMOCindLoc0 = AMOCindLoc_piC[i[1]]
    myAMOCindMax0 = AMOCindMax_piC[i[1]]
    myAMOCindLoc = np.zeros([1800])
    myAMOCindMax = np.zeros([1800])
    for j in range(0,12):
        myAMOCindLoc[j::12] = myAMOCindLoc0[j::12]-np.mean(myAMOCindLoc0[j::12])
        myAMOCindMax[j::12] = myAMOCindMax0[j::12]-np.mean(myAMOCindMax0[j::12])
    m,b=np.polyfit(timevec,myAMOCindLoc,1)
    myAMOCindLoc = myAMOCindLoc-timevec*m
    m,b=np.polyfit(timevec,myAMOCindMax,1)
    myAMOCindMax = myAMOCindMax-timevec*m
    AMOCindLoc_piC[i[1]] = myAMOCindLoc
    AMOCindMax_piC[i[1]] = myAMOCindMax

In [18]:
#Make np array, and turn into xarray dataset
AMOCindLoc = np.zeros([len(models),1800])
AMOCindMax = np.zeros([len(models),1800])
for i in enumerate(models):
    AMOCindLoc[i[0],:] = AMOCindLoc_piC[i[1]]
    AMOCindMax[i[0],:] = AMOCindMax_piC[i[1]]
    
ds_AMOCLoc = xr.Dataset({"AMOCindex":(("models","t"),AMOCindLoc)},
                        coords = {
                            "models": models,
                            "t": np.linspace(0.,1799/12,1800)
                        },)
ds_AMOCMax = xr.Dataset({"AMOCindex":(("models","t"),AMOCindMax)},
                        coords = {
                            "models": models,
                            "t": np.linspace(0.,1799/12,1800)
                        },)

os.remove("/net/aeolus/aura/hansingh/CMIP6.AMOCindex.Control.loc.062022.nc")
os.remove("/net/aeolus/aura/hansingh/CMIP6.AMOCindex.Control.max.062022.nc")
ds_AMOCLoc.to_netcdf("/net/aeolus/aura/hansingh/CMIP6.AMOCindex.Control.loc.062022.nc")
ds_AMOCMax.to_netcdf("/net/aeolus/aura/hansingh/CMIP6.AMOCindex.Control.max.062022.nc")

In [19]:
print(ds_AMOCLoc)
print(ds_AMOCMax)

<xarray.Dataset>
Dimensions:    (models: 18, t: 1800)
Coordinates:
  * models     (models) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'SAM0-UNICON'
  * t          (t) float64 0.0 0.08333 0.1667 0.25 ... 149.7 149.8 149.8 149.9
Data variables:
    AMOCindex  (models, t) float64 2.333 3.095 2.064 ... -1.72 -2.052 -4.272
<xarray.Dataset>
Dimensions:    (models: 18, t: 1800)
Coordinates:
  * models     (models) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'SAM0-UNICON'
  * t          (t) float64 0.0 0.08333 0.1667 0.25 ... 149.7 149.8 149.8 149.9
Data variables:
    AMOCindex  (models, t) float64 1.533 2.098 8.726 ... -1.725 -2.06 -4.224


In [20]:
#for same set of models, create time series of psl, hfss, hfls, tas as [model, time, lat, lon]
#note that longitude dimension needs to be in terms of [-180,180]
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

dict_piC = {}

lats = {}
lons = {}

#set up AWS filesystem access
fs_s3 = s3fs.S3FileSystem(anon=True)
myvars = ["hfls","hfss","tas","psl"]

for imodel in enumerate(models):
    print("piControl, "+imodel[1])
    for ivar in range(0,len(myvars)):
        res = catalog.search(source_id=imodel[1], variable_id=myvars[ivar],\
                             experiment_id="piControl", table_id="Amon")
        #print(res.df['zstore'][0])
        mapper = fs_s3.get_mapper(res.df['zstore'][0])
        ds = xr.open_zarr(mapper, consolidated=True)
        if ivar==0:
            try:
                lats[imodel[1]] = ds.lat
                lons[imodel[1]] = ds.lon
            except AttributeError:
                ds = ds.rename({"latitude":"lat", "longitude":"lon"})
                lats[imodel[1]] = ds.lat
                lons[imodel[1]] = ds.lon
        try:
            dict_piC[imodel[1]+"_"+myvars[ivar]]=ds[myvars[ivar]][:1800,:,:]
        except (AttributeError, KeyError):
            print("Not enough years")
            ds = ds.rename({"latitude":"lat", "longitude":"lon"})
            dict_piC[imodel[1]+"_"+myvars[ivar]]=ds[myvars[ivar]]

piControl, ACCESS-CM2
piControl, ACCESS-ESM1-5
piControl, CESM2
piControl, CESM2-FV2
piControl, CESM2-WACCM
piControl, CESM2-WACCM-FV2
piControl, CanESM5
piControl, E3SM-1-0
piControl, FGOALS-g3
piControl, GISS-E2-1-G
piControl, GISS-E2-2-G
piControl, INM-CM4-8
piControl, MIROC6
piControl, MPI-ESM-1-2-HAM
piControl, MPI-ESM1-2-HR
piControl, MPI-ESM1-2-LR
piControl, MRI-ESM2-0
piControl, SAM0-UNICON


In [34]:
#rotate grids to [-180,180]
#meshgrid of region of interest
lons_rot = {}
piC_rot = {}

for i in enumerate(models):
    print("Working on "+i[1])
    #rotate longitudes
    mylon0 = lons[i[1]]
    nlon0 = len(mylon0)
    mylon = np.zeros([nlon0])
    zeroind = int((np.abs(mylon0-180.)).argmin())
    n0to180 = nlon0-zeroind+1
    n181to360 = nlon0-n0to180
    mylon[0:nlon0-zeroind] = mylon0[zeroind:]-360.
    mylon[nlon0-zeroind:] = mylon0[0:zeroind]
    lons_rot[i[1]] = mylon
    for ivar in enumerate(myvars):
        myfield0 = dict_piC[i[1]+"_"+ivar[1]]
        myfield = np.zeros(np.shape(myfield0))
        myfield[:,:,0:nlon0-zeroind] = myfield0[:,:,zeroind:]
        myfield[:,:,nlon0-zeroind:] = myfield0[:,:,0:zeroind]
        piC_rot[i[1]+"_"+ivar[1]] = myfield
        print(np.shape(piC_rot[i[1]+"_"+ivar[1]]))

Working on ACCESS-CM2
(1800, 144, 192)
(1800, 144, 192)
(1800, 144, 192)
(1800, 144, 192)
Working on ACCESS-ESM1-5
(1800, 145, 192)
(1800, 145, 192)
(1800, 145, 192)
(1800, 145, 192)
Working on CESM2
(1800, 192, 288)
(1800, 192, 288)
(1800, 192, 288)
(1800, 192, 288)
Working on CESM2-FV2
(1800, 96, 144)
(1800, 96, 144)
(1800, 96, 144)
(1800, 96, 144)
Working on CESM2-WACCM
(1800, 192, 288)
(1800, 192, 288)
(1800, 192, 288)
(1800, 192, 288)
Working on CESM2-WACCM-FV2
(1800, 96, 144)
(1800, 96, 144)
(1800, 96, 144)
(1800, 96, 144)
Working on CanESM5
(1800, 64, 128)
(1800, 64, 128)
(1800, 64, 128)
(1800, 64, 128)
Working on E3SM-1-0
(1800, 180, 360)
(1800, 180, 360)
(1800, 180, 360)
(1800, 180, 360)
Working on FGOALS-g3
(1800, 80, 180)
(1800, 80, 180)
(1800, 80, 180)
(1800, 80, 180)
Working on GISS-E2-1-G
(1800, 90, 144)
(1800, 90, 144)
(1800, 90, 144)
(1800, 90, 144)
Working on GISS-E2-2-G
(1800, 90, 144)
(1800, 90, 144)
(1800, 90, 144)
(1800, 90, 144)
Working on INM-CM4-8
(1800, 120, 18

In [38]:
#interpolation to ROI grid

#define ROI:
latmax = 75.
latmin = 30.
lonmin = -80.
lonmax = 30.
lat_rg = np.arange(latmin,latmax+1.)
lon_rg = np.arange(lonmin,lonmax+1.)
lon_rg_mesh,lat_rg_mesh = np.meshgrid(lon_rg,lat_rg)
nlon_rg = len(lon_rg)
nlat_rg = len(lat_rg)

#regrid loop
piC_rg = {}
for i in enumerate(models):
    mylon = lons_rot[i[1]]
    mylat = lats[i[1]]
    for ivar in enumerate(myvars):
        print("Working on "+i[1]+", "+ivar[1])
        myfield = piC_rot[i[1]+"_"+ivar[1]]
        myfield_rg = np.zeros([ntime,nlat_rg,nlon_rg])
        for itime in range(0,ntime):
            myfunc = scipy.interpolate.RectBivariateSpline(mylat,mylon,myfield[itime,:,:])
            myfield_rg[itime,:,:] = myfunc(lat_rg,lon_rg)
        piC_rg[i[1]+"_"+ivar[1]] = myfield_rg

Working on ACCESS-CM2, hfls
Working on ACCESS-CM2, hfss
Working on ACCESS-CM2, tas
Working on ACCESS-CM2, psl
Working on ACCESS-ESM1-5, hfls
Working on ACCESS-ESM1-5, hfss
Working on ACCESS-ESM1-5, tas
Working on ACCESS-ESM1-5, psl
Working on CESM2, hfls
Working on CESM2, hfss
Working on CESM2, tas
Working on CESM2, psl
Working on CESM2-FV2, hfls
Working on CESM2-FV2, hfss
Working on CESM2-FV2, tas
Working on CESM2-FV2, psl
Working on CESM2-WACCM, hfls
Working on CESM2-WACCM, hfss
Working on CESM2-WACCM, tas
Working on CESM2-WACCM, psl
Working on CESM2-WACCM-FV2, hfls
Working on CESM2-WACCM-FV2, hfss
Working on CESM2-WACCM-FV2, tas
Working on CESM2-WACCM-FV2, psl
Working on CanESM5, hfls
Working on CanESM5, hfss
Working on CanESM5, tas
Working on CanESM5, psl
Working on E3SM-1-0, hfls
Working on E3SM-1-0, hfss
Working on E3SM-1-0, tas
Working on E3SM-1-0, psl
Working on FGOALS-g3, hfls
Working on FGOALS-g3, hfss
Working on FGOALS-g3, tas
Working on FGOALS-g3, psl
Working on GISS-E2-1-G

In [45]:
#apply landmask (maybe? or not?)
#remove seasonal cycle
#detrend
piC_fin_mon = {}
piC_fin_yr = {}
#landmasked version
piC_fin_mon_lm = {}
piC_fin_yr_lm = {}
monthvec = np.linspace(0.,(ntime-1)/12,ntime)
yrvec = np.linspace(0.,nyr-1,nyr)

#create landmask and put it on ROI grid
datadir = "/net/aeolus/aura/hansingh/SOMClimos/"
myfile = "SOM_Control.cam5.0030-0059.ann.nc"
deg2rad = np.pi/180.
Re = 6.4e6
cp = 3850.
rho = 1025.
myData_C = xr.open_dataset(datadir+myfile, decode_times=False)
mylat_cesm = np.array(myData_C.lat)
mylon_cesm = np.array(myData_C.lon)
landfrac = np.array(myData_C.LANDFRAC[0,:,:])
myfunc = scipy.interpolate.RectBivariateSpline(mylat,mylon,landfrac)
landfrac_rg = myfunc(lat_rg,lon_rg)
landfrac_rg = np.where(landfrac_rg<0.5,0.,landfrac_rg)
landfrac_rg = np.where(landfrac_rg>=0.5,1.,landfrac_rg)

for i in enumerate(models):
    for ivar in enumerate(myvars):
        print("Working on "+i[1]+", "+ivar[1])
        myfield0 = piC_rg[i[1]+"_"+ivar[1]]
        myfield_mon = np.zeros([ntime,nlat_rg,nlon_rg])
        myfield_yr = np.zeros([nyr,nlat_rg,nlon_rg])
        #remove seasonal cycle
        for imon in range(0,12):
            monmean = np.mean(myfield0[imon::12,:,:],axis=0)
            myfield_mon[imon::12,:,:] = myfield0[imon::12,:,:]-monmean
        #calculate annual means
        for iyr in range(0,nyr):
            myfield_yr[iyr,:,:] = np.mean(myfield0[iyr*12:(iyr+1)*12,:,:],axis=0)
        myfield_yr = myfield_yr-np.mean(myfield_yr,axis=0) #remove mean
        #detrend
        for ilat in range(0,nlat_rg):
            for ilon in range(0,nlon_rg):
                slope, yint, rval, pval, exerr = scipy.stats.linregress(monthvec,myfield_mon[:,ilat,ilon])
                myfield_mon[:,ilat,ilon] = myfield_mon[:,ilat,ilon]-monthvec*slope
                slope, yint, rval, pval, exerr = scipy.stats.linregress(yrvec,myfield_yr[:,ilat,ilon])
                myfield_yr[:,ilat,ilon] = myfield_yr[:,ilat,ilon]-yrvec*slope
        #put into dicts
        piC_fin_mon[i[1]+"_"+ivar[1]] = myfield_mon
        piC_fin_yr[i[1]+"_"+ivar[1]] = myfield_yr
        #apply ocean mask
        piC_fin_mon_lm[i[1]+"_"+ivar[1]] = myfield_mon*(1.-landfrac_rg)
        piC_fin_yr_lm[i[1]+"_"+ivar[1]] = myfield_yr*(1.-landfrac_rg)

Working on ACCESS-CM2, hfls
Working on ACCESS-CM2, hfss
Working on ACCESS-CM2, tas
Working on ACCESS-CM2, psl
Working on ACCESS-ESM1-5, hfls
Working on ACCESS-ESM1-5, hfss
Working on ACCESS-ESM1-5, tas
Working on ACCESS-ESM1-5, psl
Working on CESM2, hfls
Working on CESM2, hfss
Working on CESM2, tas
Working on CESM2, psl
Working on CESM2-FV2, hfls
Working on CESM2-FV2, hfss
Working on CESM2-FV2, tas
Working on CESM2-FV2, psl
Working on CESM2-WACCM, hfls
Working on CESM2-WACCM, hfss
Working on CESM2-WACCM, tas
Working on CESM2-WACCM, psl
Working on CESM2-WACCM-FV2, hfls
Working on CESM2-WACCM-FV2, hfss
Working on CESM2-WACCM-FV2, tas
Working on CESM2-WACCM-FV2, psl
Working on CanESM5, hfls
Working on CanESM5, hfss
Working on CanESM5, tas
Working on CanESM5, psl
Working on E3SM-1-0, hfls
Working on E3SM-1-0, hfss
Working on E3SM-1-0, tas
Working on E3SM-1-0, psl
Working on FGOALS-g3, hfls
Working on FGOALS-g3, hfss
Working on FGOALS-g3, tas
Working on FGOALS-g3, psl
Working on GISS-E2-1-G

In [48]:
for ivar in enumerate(myvars):
    mydata = np.zeros([len(models),ntime,nlat_rg,nlon_rg])
    for imod in enumerate(models):
        mydata[imod[0],:,:,:] = piC_fin_mon[imod[1]+"_"+ivar[1]]
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":models,
                       "t": monthvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.month.062022.nc")
    mydata = np.zeros([len(models),nyr,nlat_rg,nlon_rg])
    for imod in enumerate(models):
        mydata[imod[0],:,:,:] = piC_fin_yr[imod[1]+"_"+ivar[1]]
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":models,
                       "t": yrvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.annual.062022.nc")
    mydata = np.zeros([len(models),ntime,nlat_rg,nlon_rg])
    for imod in enumerate(models):
        mydata[imod[0],:,:,:] = piC_fin_mon_lm[imod[1]+"_"+ivar[1]]
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":models,
                       "t": monthvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.OcnOnly.month.062022.nc")
    mydata = np.zeros([len(models),nyr,nlat_rg,nlon_rg])
    for imod in enumerate(models):
        mydata[imod[0],:,:,:] = piC_fin_yr_lm[imod[1]+"_"+ivar[1]]
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":models,
                       "t": yrvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.OcnOnly.annual.062022.nc")
    

In [49]:
#Use netcdf files to load dOHU
dict_dOHU = {}
mydir = "/net/aeolus/aura/hansingh/CMIP6AWS_data_processed/"

for i in enumerate(models):
    imod = i[1]
    print("Working on "+imod)
    try:
        OHU_C = xr.open_dataset(mydir+"rsds."+imod+".piControl.nc").rsds-xr.open_dataset(mydir+"rsus."+imod+".piControl.nc").rsus+\
                xr.open_dataset(mydir+"rlds."+imod+".piControl.nc").rlds-xr.open_dataset(mydir+"rlus."+imod+".piControl.nc").rlus-\
                xr.open_dataset(mydir+"hfss."+imod+".piControl.nc").hfss-xr.open_dataset(mydir+"hfls."+imod+".piControl.nc").hfls
        OHU_E = xr.open_dataset(mydir+"rsds."+imod+".abrupt4XCO2.nc").rsds-xr.open_dataset(mydir+"rsus."+imod+".abrupt4XCO2.nc").rsus+\
                xr.open_dataset(mydir+"rlds."+imod+".abrupt4XCO2.nc").rlds-xr.open_dataset(mydir+"rlus."+imod+".abrupt4XCO2.nc").rlus-\
                xr.open_dataset(mydir+"hfss."+imod+".abrupt4XCO2.nc").hfss-xr.open_dataset(mydir+"hfls."+imod+".abrupt4XCO2.nc").hfls
        dict_dOHU[imod] = (OHU_E-OHU_C).squeeze()
    except:
        print("**Model not found.")
        
#Calculate OHU indices
#Calculate Heat Uptake Indices
#Calculate global mean and regional heat uptake
dict_GM_OHU_anom = {}
dict_SHext_OHU_anom = {}
dict_NHext_OHU_anom = {}
deg2rad = np.pi/180.

for i in enumerate(models):
    imod = i[1]
    try:
        mylat = np.array(lats[imod])
        print(imod)
        coslat = np.cos(deg2rad*mylat)
        coslat_SHext = np.where(mylat<-50.,coslat,0.)
        coslat_NHext = np.where(mylat>50.,coslat,0.)
        mydOHU = dict_dOHU[imod]
        dict_GM_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat)/np.nansum(coslat)
        dict_SHext_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat_SHext)/np.nansum(coslat_SHext)
        dict_NHext_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat_NHext)/np.nansum(coslat_NHext)
    except:
        print("**Not found.")

Working on ACCESS-CM2
Working on ACCESS-ESM1-5
Working on CESM2
Working on CESM2-FV2
Working on CESM2-WACCM
Working on CESM2-WACCM-FV2
Working on CanESM5
Working on E3SM-1-0
Working on FGOALS-g3
Working on GISS-E2-1-G
Working on GISS-E2-2-G
Working on INM-CM4-8
Working on MIROC6
Working on MPI-ESM-1-2-HAM
Working on MPI-ESM1-2-HR
Working on MPI-ESM1-2-LR
Working on MRI-ESM2-0
**Model not found.
Working on SAM0-UNICON
ACCESS-CM2
ACCESS-ESM1-5
CESM2
CESM2-FV2
CESM2-WACCM
CESM2-WACCM-FV2
CanESM5
E3SM-1-0
FGOALS-g3
GISS-E2-1-G
GISS-E2-2-G
INM-CM4-8
MIROC6
MPI-ESM-1-2-HAM
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
**Not found.
SAM0-UNICON


In [58]:
print(dict_NHext_OHU_anom)
dOHU_NH = np.zeros([len(models)])
for i in enumerate(models):
    try:
        dOHU_NH[i[0]] = dict_NHext_OHU_anom[i[1]]
    except:
        dOHU_NH[i[0]] = np.nan
        
print(dOHU_NH)
myds = xr.Dataset({"dOHU":(("models"),dOHU_NH)},
                  coords = {
                      "models": models
                  },)
#os.remove("/net/aeolus/aura/hansingh/CMIP6.OcnHeatUptake.4XCO2.062022.pdf")
myds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6_OHUdata/CMIP6.OcnHeatUptake.4XCO2.062022.nc")

{'ACCESS-CM2': 5.069753901720349, 'ACCESS-ESM1-5': 2.4882552462510663, 'CESM2': 8.839188644119957, 'CESM2-FV2': 9.16785190771157, 'CESM2-WACCM': 8.535441704125132, 'CESM2-WACCM-FV2': 8.869834498690738, 'CanESM5': 4.5455000859568395, 'E3SM-1-0': 3.7530974761661042, 'FGOALS-g3': 4.500549003077796, 'GISS-E2-1-G': 10.009460452837633, 'GISS-E2-2-G': 7.181215819251269, 'INM-CM4-8': 2.035252762553003, 'MIROC6': 5.9347614565180615, 'MPI-ESM-1-2-HAM': 5.033206612744432, 'MPI-ESM1-2-HR': 4.809416286484583, 'MPI-ESM1-2-LR': 4.314390270693941, 'SAM0-UNICON': 9.355502162646175}
[ 5.0697539   2.48825525  8.83918864  9.16785191  8.5354417   8.8698345
  4.54550009  3.75309748  4.500549   10.00946045  7.18121582  2.03525276
  5.93476146  5.03320661  4.80941629  4.31439027         nan  9.35550216]
